## Tratamiento de fichero y exportación de resultado a Mysql

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

scSpark = SparkSession \
        .builder \
        .appName("Ejemplo ETL") \
        .getOrCreate()

In [0]:
scSpark

In [0]:
%sh
wget https://raw.githubusercontent.com/jgomezcuadrado/datsets/master/bank-full.csv -O bank.csv

In [0]:
%fs
ls /FileStore

In [0]:
data_file = 'dbfs:/FileStore/sabadell/supermarket_sales.csv'
sdfData = scSpark.read.csv(data_file, header=True, sep=",").cache()
print('Total Records = {}'.format(sdfData.count()))
sdfData.show()

### Hacer un groupBy()

In [0]:
gender = sdfData.groupBy('Gender').count()
print(gender.show())

## Usando la API de SQL
* Crear una tabla temporal usando registerTampTable. 
* Ya se pueden hacer consultas SQL sobre la tabla creada

In [0]:
sdfData.registerTempTable("sales")
output =  scSpark.sql('SELECT * from sales')
display(output)

In [0]:
output = scSpark.sql('SELECT * from sales WHERE `Unit Price` < 15 AND Quantity < 10')
display(output)

In [0]:
output = scSpark.sql('SELECT COUNT(*) as total, City from sales GROUP BY City')
output.show()

In [0]:
%sql


### Se pueden guardar los resultados en otro fichero para persistir

In [0]:
output.write.mode("overwrite").format('csv').save('dbfs:/FileStore/sabadell/filtrados')

- Creó una carpeta con el nombre del archivo (filtrados). 
- Dentro un archivo con el nombre _SUCCESS dice si la operación fue exitosa o no. 
- En caso de que falle, se genera un archivo con el nombre _FAILURE. 
- Debería haber varios archivos PART ya que cada job está involucrado en la operación de escritura en el archivo. 
- Si se quiere crear un solo archivo, se puede usar coalesce que recopila y reduce los datos de todas las particiones a un solo marco de datos.

In [0]:
output.coalesce(1).write.mode("overwrite").format('json').save('dbfs:/FileStore/sabadell/filtrados2')


Hacemos uso de los comandos del sistema de archivos

In [0]:
%fs 
ls dbfs:/FileStore/sabadell/filtrados/


In [0]:
%fs
head dbfs:/FileStore/sabadell/filtrados/part-00000-tid-3612515660995380277-88bc8fe3-f49e-4e43-80ee-2cbec4d2fbf9-32-1-c000.csv

In [0]:
#scSpark = SparkSession \
#        .builder \
#        .appName("reading csv") \
#        .config("spark.driver.extraClassPath", "/usr/local/spark/jars/mysql-connector-java-8.0.16.jar") \
#        .getOrCreate()

In [0]:
output = scSpark.sql('SELECT COUNT(*) as total, City from sales GROUP BY City')
#output.show()


In [0]:
output.write.format('jdbc').options(
        url='jdbc:mysql://sql11.freesqldatabase.com/sql11453803',
        driver='com.mysql.cj.jdbc.Driver',
        dbtable='city_info',
        user='sql11453803',
        password='bT9yTpIyGh').mode('append').save()